In [87]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Normalization
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from keras import layers
pd.set_option('display.max_rows', 50)
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping 
from tensorflow.keras.callbacks import ModelCheckpoint 


import random


In [2]:
df_merged = pd.read_csv("rnn_model_df_31.csv")

In [3]:
df_merged = df_merged[df_merged["reply_count"].notna()].drop(columns = ["reply_count", "avg_followers_count", "avg_following_count" ] )

In [4]:
df_merged

,tweet_date,party,poll,retweet_count,like_count,avg_len_of_tweet,avg_user_tweet_count,avg_ff_ratio,share_of_tweets,share_unique_users,weighted_sentiment,share_of_positive_tweets,share_of_negative_tweets
420,2021-07-23,AFD,11.00,7430.0,50589.0,183.483988,30258.523792,11.717405,0.241723,0.599421,-0.822698,0.028768,0.587842
421,2021-07-23,CDU,28.00,8987.0,53965.0,191.227455,26716.583105,5.530039,0.207085,0.673073,-0.466696,0.035058,0.480465
422,2021-07-23,FDP,12.00,1854.0,15224.0,171.541570,26521.715935,5.561241,0.113623,0.675905,-0.385271,0.061971,0.476905
423,2021-07-23,GRUENE,19.00,7621.0,60041.0,182.306444,24578.680470,4.759415,0.264028,0.619016,-0.267626,0.060460,0.439125
424,2021-07-23,LINKE,7.00,268.0,1625.0,181.312292,36722.935216,2.647902,0.026328,0.616279,-0.050804,0.061462,0.373754
...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,2021-08-31,FDP,12.00,2626.0,30829.0,148.846072,20007.450027,3.306954,0.112983,0.721005,-0.545316,0.051131,0.523784
696,2021-08-31,GRUENE,17.50,4733.0,38891.0,180.245516,23105.896824,6.798644,0.122323,0.688827,-0.115527,0.063847,0.393780
697,2021-08-31,LINKE,6.50,1411.0,10948.0,176.158121,27386.891000,2.613547,0.056139,0.547508,-0.117520,0.057727,0.389745
698,2021-08-31,OTHER,7.67,1580.0,9130.0,164.704657,16861.637255,2.676769,0.032850,0.670343,0.000828,0.088848,0.371936


In [5]:
df_merged = df_merged.replace(np.nan, 0)

In [6]:
df_merged["tweet_date"] = pd.to_datetime(df_merged["tweet_date"])

df_merged = df_merged.groupby([pd.Grouper(key='tweet_date',freq='D'), "party"]).agg({
#     "reply_count": "sum", 
    "retweet_count": "sum",
    "like_count": "sum",
#     "avg_len_of_tweet": "mean" ,
#     "avg_followers_count": "mean" ,
#     "avg_following_count": "mean" ,
    "avg_user_tweet_count": "mean",
    "avg_ff_ratio" :"mean",
    "share_of_tweets" :"mean",
    "share_unique_users": "mean",
    "weighted_sentiment" : "mean",
    "share_of_positive_tweets": "mean",
    "share_of_negative_tweets" : "mean",
    "poll": "mean"}) 

In [7]:
df_real_true = df_merged.iloc[-7:]

In [8]:
df_merged = df_merged.iloc[:-7]

In [9]:
df_merged_final_test = df_merged.iloc[-7:]

In [43]:
df_merged_final = df_merged.iloc[:-7]

In [44]:
num_features = [ "retweet_count", "like_count", "avg_user_tweet_count","avg_ff_ratio"]
remaing_columns = ["share_of_tweets","share_unique_users","weighted_sentiment","share_of_positive_tweets","share_of_negative_tweets"]

In [45]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()

mm_scaler.fit(df_merged_final[num_features])

numerical_train_scaled = mm_scaler.transform(df_merged_final[num_features])
numerical_test_scaled = mm_scaler.transform(df_merged_final_test[num_features])

In [46]:
df_merged_poll = df_merged_final["poll"] / 100
df_merged_poll_test = df_merged_final_test["poll"] / 100

In [47]:
df_numerical_train_scaled = pd.DataFrame(numerical_train_scaled, columns = num_features, index = df_merged_final.index)


In [48]:
df_train_scaled = pd.concat([df_numerical_train_scaled,df_merged_final[remaing_columns], pd.DataFrame(df_merged_poll, columns = ["poll"], index = df_merged_final.index)] , axis = 1)

In [49]:
df_test_scaled = pd.concat([pd.DataFrame(numerical_test_scaled, columns = num_features, index = df_merged_final_test.index),df_merged_final_test[remaing_columns], pd.DataFrame(df_merged_poll_test, columns = ["poll"], index = df_merged_final_test.index)] , axis = 1)

In [50]:
def subsequence(df, length, start = 0):
    last_possible_start = len(df) - length 
    X = df[start:start + length]
    y = df["poll"][(start + length ) : (start + length + 7)]
    
    return X, y

In [51]:
def multiple_subsequences(df, length):
    
    list_of_X = []
    list_of_y = []
    
    number_of_subsequences = round(len(df) /  7 - (length/7))
    print(number_of_subsequences)
    start = 0
    for num in range(number_of_subsequences):
        temporary_X, temporary_y = subsequence(df, length, start)
        list_of_X.append(temporary_X)
        list_of_y.append(temporary_y)
        start = start + 7
    
    return list_of_X, list_of_y

    
    
    

In [271]:
X_train , y_train = multiple_subsequences(pd.DataFrame(df_train_scaled), 14 )
# X_test , y_test = multiple_subsequences(df_test_scaled, 35)

36


In [272]:
(df_train_scaled.shape[0]/ 7)

38.0

In [273]:
X_train = np.array(X_train)
# X_test = np.array(X_test)
y_train = np.array(y_train)
# y_test = np.array(y_test)


In [274]:
from keras.layers import BatchNormalization


In [284]:
model = Sequential()
# model.add(layers.Masking(mask_value=-9999, input_shape=X_train[0].shape))
model.add(layers.LSTM(units=128, activation='tanh', return_sequences=True))
model.add(layers.LSTM(units=64, activation='tanh', return_sequences=False))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(7, activation='linear'))


In [285]:
%%time

es = EarlyStopping(  monitor='val_mae', patience = 40, restore_best_weights=True)
mcp_save = ModelCheckpoint('.mdl_wts.tf', save_best_only=True, monitor='val_loss', mode='min', verbose = 1)



model.compile(loss='mse', optimizer='adam', metrics=['mae'])

history = model.fit(X_train, y_train ,
          epochs=100, 
          batch_size=1, 
          verbose=1, 
          validation_split=0.1,
          shuffle = False,
          callbacks=[es]
     )



Epoch 1/100
32/32 [==============================] - 3s 30ms/step - loss: 0.0038 - mae: 0.0393 - val_loss: 2.2274e-04 - val_mae: 0.0133
Epoch 2/100
32/32 [==============================] - 0s 11ms/step - loss: 3.6542e-04 - mae: 0.0149 - val_loss: 1.3966e-04 - val_mae: 0.0096
Epoch 3/100
32/32 [==============================] - 0s 10ms/step - loss: 2.7044e-04 - mae: 0.0127 - val_loss: 1.3593e-04 - val_mae: 0.0100
Epoch 4/100
32/32 [==============================] - 0s 10ms/step - loss: 3.0263e-04 - mae: 0.0132 - val_loss: 1.7664e-04 - val_mae: 0.0119
Epoch 5/100
32/32 [==============================] - 0s 10ms/step - loss: 3.2781e-04 - mae: 0.0138 - val_loss: 1.2001e-04 - val_mae: 0.0092
Epoch 6/100
32/32 [==============================] - 0s 11ms/step - loss: 3.0156e-04 - mae: 0.0131 - val_loss: 1.1028e-04 - val_mae: 0.0087
Epoch 7/100
32/32 [==============================] - 0s 11ms/step - loss: 2.8490e-04 - mae: 0.0128 - val_loss: 1.0885e-04 - val_mae: 0.0087
Epoch 8/100
32/32 [=====

32/32 [==============================] - 1s 21ms/step - loss: 9.6429e-05 - mae: 0.0074 - val_loss: 1.6897e-04 - val_mae: 0.0090
Epoch 60/100
32/32 [==============================] - 0s 13ms/step - loss: 9.6647e-05 - mae: 0.0074 - val_loss: 1.6952e-04 - val_mae: 0.0091
Epoch 61/100
32/32 [==============================] - 0s 16ms/step - loss: 9.6412e-05 - mae: 0.0073 - val_loss: 1.6927e-04 - val_mae: 0.0090
Epoch 62/100
32/32 [==============================] - 0s 13ms/step - loss: 9.6924e-05 - mae: 0.0075 - val_loss: 1.7477e-04 - val_mae: 0.0092
Epoch 63/100
32/32 [==============================] - 0s 13ms/step - loss: 9.7788e-05 - mae: 0.0075 - val_loss: 1.7675e-04 - val_mae: 0.0093
Epoch 64/100
32/32 [==============================] - 0s 13ms/step - loss: 9.7527e-05 - mae: 0.0075 - val_loss: 1.7932e-04 - val_mae: 0.0094
Epoch 65/100
32/32 [==============================] - 0s 13ms/step - loss: 9.8601e-05 - mae: 0.0075 - val_loss: 1.7677e-04 - val_mae: 0.0094
Epoch 66/100
32/32 [======

In [286]:
df_merged_final_day = np.array(df_test_scaled)
df_merged_final_day = np.expand_dims(df_merged_final_day, axis = 0)

In [287]:
df_prediction = pd.DataFrame(model.predict(df_merged_final_day)).rename( \
                        columns = {0:'AFD',1:'CDU',2:'FDP',3: 'GRUENE',4: 'LINKE', 5:'OTHER', 6: 'SPD'}).round(3)
df_prediction.index = [df_real_true.index[0][0]]

df_prediction

,AFD,CDU,FDP,GRUENE,LINKE,OTHER,SPD
2021-08-31,0.089,0.188,0.105,0.146,0.064,0.057,0.191


In [280]:
df_prediction_2 = pd.DataFrame((df_prediction.iloc[0]) / np.sum(df_prediction.iloc[0])).T
df_prediction_2 = df_prediction_2.round(2)

In [281]:
df_prediction_2

,AFD,CDU,FDP,GRUENE,LINKE,OTHER,SPD
2021-08-31,-0.04,0.2,0.22,0.15,0.13,0.12,0.23


In [282]:
df_prediction_2.to_csv("final_real_dataframe_for_philip.csv")

In [283]:
df_true = pd.DataFrame(df_real_true["poll"]).T/100

In [234]:
df_true = pd.DataFrame(df_true.iloc[0]).reset_index().groupby(by = "party").mean().T

In [235]:
df_true

party,AFD,CDU,FDP,GRUENE,LINKE,OTHER,SPD
poll,0.1083,0.205,0.12,0.175,0.065,0.0767,0.25


In [ ]:
df_plotting_spd = df_train_scaled.reset_index().loc[df_train_scaled.reset_index()["party"] == "SPD"].drop(columns = "party")\
[["tweet_date","share_of_tweets", "weighted_sentiment", "share_of_positive_tweets", "share_of_negative_tweets", "poll"]]
df_plotting_cdu = df_train_scaled.reset_index().loc[df_train_scaled.reset_index()["party"] == "CDU"].drop(columns = "party")\
[["tweet_date","share_of_tweets", "weighted_sentiment", "share_of_positive_tweets", "share_of_negative_tweets", "poll"]]


In [ ]:
sns.set(rc={'figure.figsize':(15,10)})

new_df = df_plotting_spd.melt(id_vars='tweet_date', 
                 value_name='val', 
                 var_name='type')

sns.lineplot(x='tweet_date', y='val', hue='type', data=new_df)

In [ ]:
new_df = df_plotting_cdu.melt(id_vars='tweet_date', 
                 value_name='val', 
                 var_name='type')

sns.lineplot(x='tweet_date', y='val', hue='type', data=new_df)

In [ ]:
df_merged_final_test

In [ ]:
5+5